In [1]:
import pandas as pd
import numpy as np
import surpyval as surv
import matplotlib.pyplot as plt
import itertools
from tqdm import tqdm
from reliability.Distributions import Weibull_Distribution

In [2]:
# Definiendo funciones

# Rectangularización
def fill_in(df, id_cols):
    """Fill in empty records for combinations of id_cols that do not exist
    in dataset.
    
    Args:
        df: dataset
        id_cols: list of identity columns

    Returns:
        filled_df: dataframe with empty records for missing combinations of id_cols
    """
    # create all possible unique combinations of id_cols
    # and find combos that do not exist in the dataset
    id_combos = list(itertools.product(*[df[c].unique() for c in id_cols]))
    existing_combos = df[id_cols].apply(tuple, axis=1).unique()
    missing_combos = set(id_combos) - set(existing_combos)

    # create an empty dataframe with the missing combos
    other_cols = [c for c in df.columns if c not in id_cols]
    new_idx = pd.MultiIndex.from_tuples(missing_combos, names=id_cols)
    empty_data = np.empty(shape=(len(missing_combos), len(other_cols))).fill(np.nan)
    filled_df = pd.DataFrame(data=empty_data, index=new_idx, columns=other_cols).reset_index()
    return pd.concat([df.assign(_fill_in=0), filled_df.assign(_fill_in=1)]) 

# Imputación hotdeck
def hotdeck_imputation(row):
    '''
    Imputa valores aleatorios a la base del Seguro de Cesantía a partir de la Encuesta Suplementaria de Ingresos
    de acuerdo a subgrupos poblacionales.
    '''
    if pd.isna(row.remp_imp_t):
    #f row.remp_imp_t == 0:   
        #fila_elegida = df_esi.query('sexo == @row.sexo').query('grupo_etario == @row.grupo_etario').query('ano == @row.ano').sample(n=1, weights = df_esi.fact_cal_esi).iloc[0] #
        fila_elegida = df_esi.query('sexo == @row.sexo').query('grupo_etario == @row.grupo_etario').query('ano == @row.ano').sample(n=1).iloc[0]
        row.remp_imp_t = fila_elegida.ing_t_t
        row['busc_trab'] = fila_elegida.e2
        row['imputado'] = True
    else:
        row['imputado'] = False
    return row

# Hotdeck + Weibull
def tope_en_hotdeck(row):
    '''
    Imputa ingresos censurados de acuerdo a distribución Weibull en base de dato con lagunas imputadas por
    hotdeck.
    '''
    if (row.rem_tope) == 1:
        valor_imputado = df_topes_imputados.query('rem_tope == 1').sample(n=1).iloc[0].remp_imp_t
        row.remp_imp_t = valor_imputado
        row['imputado_tope'] = True
    else:
        row['imputado_tope'] = False
    return row

# Cálculo Gini
def gini(array):
    '''
    Calcula el índice Gini a partir de un np.array
    '''
    #Se pasa a numpy
    array = array.to_numpy()
    array = array.flatten()
    if np.amin(array) < 0:
        # Values cannot be negative:
        array -= np.amin(array)
    # Values cannot be 0:
    array += 0.0000001
    # Values must be sorted:
    array = np.sort(array)
    # Index per array element:
    index = np.arange(1,array.shape[0]+1)
    # Number of array elements:
    n = array.shape[0]
    # Gini coefficient:
    return ((np.sum((2 * index - n  - 1) * array)) / (n * np.sum(array)))

# Ratios interdeciles
def r80_20(series):
    top20, bottom20 = series.quantile([.8, .2])
    r = top20/bottom20
    return r

def r90_10(series):
    top10, bottom10 = series.quantile([.9, .1])
    r = top10/bottom10
    return r

# Obtener tabla con indicadores de los distintos DFs
def get_indicators(df):
    '''
    Crea una tabla con indicadores de desigualdad a partir de 
    un dataframe.
    '''
    data = []

    for fecha_devengamiento_remuneracion, ae_df in df.groupby('fecha_devengamiento_remuneracion'):
        row = [
            fecha_devengamiento_remuneracion, 
            round(gini(ae_df.remp_imp_t), ndigits = 3), 
            round(r80_20(ae_df.remp_imp_t), ndigits = 3),
            round(r90_10(ae_df.remp_imp_t), ndigits = 3),
        ]
        data.append(row)
        
    col_header = ['Mes', 'Gini', 'P80/P20', 'P90/10']
    tabla = pd.DataFrame(data, columns = col_header)
    return tabla

In [3]:
#Nombrando columnas

colnames_afiliados = [
    'id_persona_afiliados', 
    'sexo', 
    'fecha_nacimiento', 
    'nivel_educacional', 
    'anos_educacion', 
    'estado_civil', 
    'comuna_domicilio', 
    'institucion_previsional', 
    'nacionalidad'
] 

colnames_cuentas = [
    'id_persona_cuentas', 
    'tipo_afiliacion', 
    'lugar_afiliacion', 
    'fecha_afiliacion', 
    'codigo_estado', 
    'modalidad_afiliacion', 
    'saldo_cuotas', 
    'saldo_cero'
]

colnames_rentas = [
    'id_persona_rentas', 
    'fecha_devengamiento_remuneracion', 
    'tipo_contrato', 
    'subsidio_incapacidad', 
    'actividad_economica', 
    'comuna_empleador', 
    'ingreso_imponible', 
    'n_trabajadores_empresa', 
    'renta_promedio_empresa', 
    'de_renta_promedio_empresa', 
    'ingreso_imponible_0', 
    'ingreso_imponible_tope', 
    'ingreso_minimo', 
    'id_empleador'
]  

# Importando Bases.
# Nótese que por motivos computacionales se está trabajando con una muestra del 3%.

afiliados3pp = pd.read_csv(
    r'C:\Users\alfre\OneDrive\Documentos\Horizontal\BDSC\Datos\3pp\1_afiliados.csv', 
    names = colnames_afiliados, 
    header = None, 
    delimiter = ';'
)

cuentas3pp = pd.read_csv(
    r'C:\Users\alfre\OneDrive\Documentos\Horizontal\BDSC\Datos\3pp\2_cuentas.csv', 
    names = colnames_cuentas, 
    header = None, 
    delimiter = ';'
)

rentas_imponibles3pp = pd.read_csv(
    r'C:\Users\alfre\OneDrive\Documentos\Horizontal\BDSC\Datos\3pp\5_rentas_imponibles.csv', 
    names = colnames_rentas, 
    header = None, 
    delimiter = ';'
)

In [4]:
# Haciendo merge entre las tres bases
afiliados_cuentas3pp = pd.merge(
    afiliados3pp, 
    cuentas3pp, 
    how='inner', 
    left_on=['id_persona_afiliados'], 
    right_on=['id_persona_cuentas']
)

bdsc3pp = pd.merge(
    rentas_imponibles3pp, 
    afiliados_cuentas3pp, 
    how='inner', 
    left_on=['id_persona_rentas'], 
    right_on=['id_persona_afiliados']
)

# Definiendo qué columnas se mantendrán
cols_keep = [
    'id_persona_afiliados', 
    'sexo', 
    'fecha_nacimiento', 
    'fecha_devengamiento_remuneracion', 
    'ingreso_imponible', 
    'ingreso_imponible_tope'
]

bdsc3pp = bdsc3pp[cols_keep]

# Renombrando columnas
bdsc3pp = bdsc3pp.rename(columns={
    'ingreso_imponible': 'remp_imp', 
    'ingreso_imponible_tope': 'rem_tope', 
    'id_persona_afiliados': 'id_pers'
})

# Definiendo periodo de análisis
bdsc3pp = bdsc3pp.query('fecha_devengamiento_remuneracion > 201908 & fecha_devengamiento_remuneracion < 201912')
bdsc3pp['ano'] = round(bdsc3pp.fecha_devengamiento_remuneracion / 100)
df_bdsc = bdsc3pp

#df_bdsc = df_bdsc.sample(n=500000)

In [8]:
# Ahora se busca identificar a cotizantes con dos o más cotizaciones al mes
# y compactarlos en una sola observación mensual.

# Se genera una variable que incorpora la renta total de los trabajadores al mes
df_bdsc['remp_imp_t'] = df_bdsc.groupby(['id_pers', 'fecha_devengamiento_remuneracion'])['remp_imp'].transform('sum')
df_bdsc.remp_imp_t = df_bdsc.remp_imp_t.replace(0, np.nan)

# Se identifica a cotizantes repetidos en el mes y se elige su primera observación
df_bdsc['counter'] = df_bdsc.groupby(['id_pers', 'fecha_devengamiento_remuneracion']).cumcount() + 1
df_bdsc = df_bdsc.query('counter < 2')

In [12]:
# Se recodifica la variable sexo para concordancia con encuesta ESI
df_bdsc['sexo_n'] = None

df_bdsc['sexo_n'] = np.where(
   (df_bdsc['sexo'] == 'M'), 1, df_bdsc['sexo_n']
   )
df_bdsc['sexo_n'] = np.where(
   (df_bdsc['sexo'] == 'F'), 2, df_bdsc['sexo_n']
   )

df_bdsc = df_bdsc.drop(axis = 1, columns = 'sexo')
df_bdsc = df_bdsc.rename(columns = {'sexo_n': 'sexo'})

In [13]:
# Ordenando base de acuerdo a id de afiliado y fecha de remuneración:
df_bdsc = df_bdsc.sort_values(['id_pers', 'fecha_devengamiento_remuneracion'])

# Se rellenan los NA obtenidos de la rectangularización en aquellas variables en que es factible
columns_fill = ['fecha_nacimiento', 'sexo', 'ano']
for column in columns_fill:
    df_bdsc[column] = df_bdsc[column].fillna(method='ffill')
    


In [15]:
# Se eliminan el resto de NaNs (aunque no hay/debiese haber)
df_bdsc = df_bdsc.dropna(subset = ['fecha_nacimiento', 'sexo', 'ano'])

In [18]:
# Cargando ESI y seleccionando no cotizantes
cols_keep_esi20 = ['ing_t_t', 'sexo', 'edad', 'fact_cal_esi', 'e2']
cols_keep_esi18 = ['ing_t_t', 'sexo', 'edad', 'fact_per', 'e2']

df_esi20 = pd.read_csv(r'C:\Users\alfre\OneDrive\Documentos\Horizontal\BDSC\Datos\Raw\esi-2020---personas.csv',
                       encoding='latin-1')
df_esi20_nc = df_esi20.query('a1 == 2 or b5 == 1 or b7a_1 == 2 or b7a_3 == 2 or b14_rev4cl_caenes == 15 or ocup_form == 2 or sector == 2') #Seleccionando no cotizantes
df_esi20_nc = df_esi20_nc[cols_keep_esi20] #Conservando solo columnas relevantes
df_esi20_nc['ano'] = 2020

df_esi19 = pd.read_csv(r'C:\Users\alfre\OneDrive\Documentos\Horizontal\BDSC\Datos\Raw\esi-2019---personas.csv', 
                       encoding='latin-1')
df_esi19_nc = df_esi19.query('a1 == 2 or b8 == 2 or b7a_1 == 2 or b7a_3 == 2 or b14_rev4cl_caenes == 15 or ocup_form == 2 or sector == 2')
df_esi19_nc = df_esi19_nc[cols_keep_esi20]
df_esi19_nc['ano'] = 2019

df_esi18 = pd.read_csv(r'C:\Users\alfre\OneDrive\Documentos\Horizontal\BDSC\Datos\Raw\esi-2018---personas.csv')
df_esi18_nc = df_esi18.query('a1 == 2 or b8 == 2 or b7a_1 == 2 or b7a_3 == 2 or b14_rev4cl_caenes == 15 or ocup_form == 2 or sector == 2')
df_esi18_nc = df_esi18_nc[cols_keep_esi18]
df_esi18_nc = df_esi18_nc.rename(columns={'fact_per': 'fact_cal_esi'}) #renombrando columna factor expansión
df_esi18_nc['ano'] = 2018

df_esi17 = pd.read_csv(r'C:\Users\alfre\OneDrive\Documentos\Horizontal\BDSC\Datos\Raw\esi-2017---personas.csv',
                      encoding='latin-1')
df_esi17_nc = df_esi17.query('a1 == 2 or b8 == 2 or b7a_1 == 2 or b7a_3 == 2 or b14_rev4cl_caenes == 15 or ocup_form == 2 or sector == 2')
df_esi17_nc = df_esi17_nc[cols_keep_esi18]
df_esi17_nc = df_esi17_nc.rename(columns={'fact_per': 'fact_cal_esi'}) #renombrando columna factor expansión
df_esi17_nc['ano'] = 2017

frames_esi = [df_esi20_nc, df_esi19_nc, df_esi18_nc, df_esi17_nc]
df_esi = pd.concat(frames_esi, ignore_index = True)

<ipython-input-18-413a59586946>:5: DtypeWarning: Columns (151,168,236,350) have mixed types. Specify dtype option on import or set low_memory=False.
  df_esi20 = pd.read_csv(r'C:\Users\alfre\OneDrive\Documentos\Horizontal\BDSC\Datos\Raw\esi-2020---personas.csv',
<ipython-input-18-413a59586946>:11: DtypeWarning: Columns (39,79,184,297) have mixed types. Specify dtype option on import or set low_memory=False.
  df_esi19 = pd.read_csv(r'C:\Users\alfre\OneDrive\Documentos\Horizontal\BDSC\Datos\Raw\esi-2019---personas.csv',
<ipython-input-18-413a59586946>:17: DtypeWarning: Columns (39,80,133,234) have mixed types. Specify dtype option on import or set low_memory=False.
  df_esi18 = pd.read_csv(r'C:\Users\alfre\OneDrive\Documentos\Horizontal\BDSC\Datos\Raw\esi-2018---personas.csv')
<ipython-input-18-413a59586946>:23: DtypeWarning: Columns (39,297) have mixed types. Specify dtype option on import or set low_memory=False.
  df_esi17 = pd.read_csv(r'C:\Users\alfre\OneDrive\Documentos\Horizontal

In [19]:
#Creando grupos etarios esi
df_esi['grupo_etario'] = 0

df_esi['grupo_etario'] = np.where(
   (df_esi['edad'] <=17) & (df_esi['edad'] >= 15) , 1, df_esi['grupo_etario']
   )
df_esi['grupo_etario'] = np.where(
   (df_esi['edad'] <=25) & (df_esi['edad'] >= 18) , 2, df_esi['grupo_etario']
   )
df_esi['grupo_etario'] = np.where(
   (df_esi['edad'] <=45) & (df_esi['edad'] >= 26) , 3, df_esi['grupo_etario']
   )
df_esi['grupo_etario'] = np.where(
   (df_esi['edad'] >= 46) , 4, df_esi['grupo_etario']
   )

df_esi = df_esi.query('grupo_etario != 0')

#Creando edad y grupos etarios en BDSC
df_bdsc["edad"] = df_bdsc.fecha_devengamiento_remuneracion- df_bdsc.fecha_nacimiento

df_bdsc["edad"] = round(df_bdsc.edad / 100, ndigits = None)

df_bdsc['grupo_etario'] = 0

df_bdsc['grupo_etario'] = np.where(
   (df_bdsc['edad'] <=17) & (df_bdsc['edad'] >= 15) , 1, df_bdsc['grupo_etario']
   )
df_bdsc['grupo_etario'] = np.where(
   (df_bdsc['edad'] <=25) & (df_bdsc['edad'] >= 18) , 2, df_bdsc['grupo_etario']
   )
df_bdsc['grupo_etario'] = np.where(
   (df_bdsc['edad'] <=45) & (df_bdsc['edad'] >= 26) , 3, df_bdsc['grupo_etario']
   )
df_bdsc['grupo_etario'] = np.where(
   (df_bdsc['edad'] >= 46) , 4, df_bdsc['grupo_etario']
   )

df_bdsc = df_bdsc.query('grupo_etario != 0')

In [20]:
# Chequeando posible necesidad de limpieza
print(df_bdsc.rem_tope.value_counts())
print(df_bdsc.remp_imp_t.isnull().sum())

#Limpiando
df_bdsc = df_bdsc[df_bdsc.fecha_nacimiento != 0] #Drop nacimientos en 0
df_bdsc = df_bdsc[df_bdsc.sexo != "X"] #Drop sexo no identificado
#df_bdsc = df_bdsc.query('remp_imp_t != 0') #Sacamos cotizaciones por $0

# Chequeando limpieza
print(df_bdsc.sexo.value_counts())
print(df_bdsc.grupo_etario.value_counts())
print(df_esi.grupo_etario.value_counts())

0    433355
1     12596
Name: rem_tope, dtype: int64
37267
1    296936
2    185732
Name: sexo, dtype: int64
3    273634
4    145812
2     63222
Name: grupo_etario, dtype: int64
4    111517
3     51229
2     33480
1     15712
Name: grupo_etario, dtype: int64


In [21]:
# Eliminando ingresos NA para el cálculo de desigualdad 
df_bdsc_dropna = df_bdsc.dropna(subset = ['remp_imp_t']).copy()
df_bdsc_dropna = df_bdsc_dropna.dropna(subset = ['rem_tope'])

In [22]:
# 
df_bdsc_valor_remtope = df_bdsc_dropna.query('rem_tope == 1').copy()
df_bdsc_valor_remtope = df_bdsc_valor_remtope.groupby('fecha_devengamiento_remuneracion')['remp_imp_t'].agg(pd.Series.mode).reset_index()
df_bdsc_valor_remtope

,fecha_devengamiento_remuneracion,remp_imp_t
0,201909,3.33558e+06
1,201910,3.337e+06
2,201911,3.35546e+06


In [25]:
# Imputación de valores con tope imponible

# Fit distribución Weibull mensual
df_topes_imputados = pd.DataFrame()

for mes, group in df_bdsc_dropna.groupby('fecha_devengamiento_remuneracion'):
    censored_monthly_tot = group['rem_tope'].sum()#Cantidad de censurados
    censored_monthly = group.rem_tope #Censurados
    cut_off_monthly = df_bdsc_valor_remtope.query('fecha_devengamiento_remuneracion =='+ str(mes)).remp_imp_t
    print(cut_off_monthly)
    model = surv.Weibull.fit(group.remp_imp_t, censored_monthly)
    dist_monthly = Weibull_Distribution(alpha = model.params[0], beta = model.params[1])

    for i in range(len(group)):
        if group['rem_tope'].iloc[i] == 1:
            imp_random = dist_monthly.random_samples(1)
            while float(imp_random) < float(cut_off_monthly):
                imp_random = dist_monthly.random_samples(1)
            group.remp_imp_t.iloc[i] = imp_random
    print(mes)
    df_i = pd.DataFrame(group)
    df_topes_imputados = pd.concat([df_topes_imputados, df_i])

0   3.33558e+06
Name: remp_imp_t, dtype: float64


Precision was lost, try:
- Using alternate fitting method
- visually checking model fit
- change data to be closer to 1.
<ipython-input-25-ffc5858f3f53>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group.remp_imp_t.iloc[i] = imp_random


201909
1   3.337e+06
Name: remp_imp_t, dtype: float64


<ipython-input-25-ffc5858f3f53>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group.remp_imp_t.iloc[i] = imp_random


201910
2   3.35546e+06
Name: remp_imp_t, dtype: float64


Precision was lost, try:
- Using alternate fitting method
- visually checking model fit
- change data to be closer to 1.
<ipython-input-25-ffc5858f3f53>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group.remp_imp_t.iloc[i] = imp_random


201911


In [26]:
# Aplicando Hotdeck
tqdm.pandas(desc="my bar!")
df_bdsc_imputadahd = df_bdsc.progress_apply(hotdeck_imputation, axis = 1)

my bar!: 100%|████████████████████████████████████████████████████████████████| 482668/482668 [13:14<00:00, 607.53it/s]


In [27]:
print(df_bdsc_imputadahd.imputado.value_counts())
df_bdsc.query('remp_imp_t == 0').head()

False    445444
True      37224
Name: imputado, dtype: int64


,id_pers,fecha_nacimiento,fecha_devengamiento_remuneracion,remp_imp,...,counter,sexo,edad,grupo_etario


In [32]:
# Se elimina a quienes no tienen salario ni buscan trabajo
df_bdsc_imputadahd = df_bdsc_imputadahd.drop(df_bdsc_imputadahd[(df_bdsc_imputadahd.remp_imp_t == 0) & (df_bdsc_imputadahd.busc_trab == 2)].index)

In [33]:
# Se agregan los topes imputados en la base con lagunas imputadas
df_hotdeck_tope = df_bdsc_imputadahd.progress_apply(tope_en_hotdeck, axis = 1)

my bar!: 100%|████████████████████████████████████████████████████████████████| 465835/465835 [08:07<00:00, 955.56it/s]


In [34]:
tabla_sin_imputacion = get_indicators(df_bdsc_dropna)
tabla_sin_imputacion

,Mes,Gini,P80/P20,P90/10
0,201909,0.42,3.272,6.354
1,201910,0.421,3.045,6.239
2,201911,0.429,3.16,7.054


In [35]:
tabla_topes_imputados = get_indicators(df_topes_imputados)
tabla_topes_imputados

,Mes,Gini,P80/P20,P90/10
0,201909,0.43,3.272,6.354
1,201910,0.43,3.045,6.239
2,201911,0.439,3.16,7.054


In [36]:
tabla_topes_lagunas_imputados = get_indicators(df_hotdeck_tope)
tabla_topes_lagunas_imputados

,Mes,Gini,P80/P20,P90/10
0,201909,0.44,3.301,7.412
1,201910,0.438,3.139,7.1
2,201911,0.444,3.232,7.647
